# Demonstration of the Metrics To-Date

For a complete list of metrics and their documentation, please see the API Metrics [documentation](../API/simulation_api.md#metrics-computation).

This demonstration will rely on the results produced in the "How To" notebook.

In [1]:
from pprint import pprint
from pathlib import Path

import pandas as pd

from wombat.core import Simulation, Metrics
from wombat.core.library import load_yaml

pd.set_option("display.float_format", '{:,.2f}'.format)
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)

## Setup

The simulations from the How To notebook are going to be rerun as it is not recommended to create a Metrics class from scratch due to the
large number of inputs that are required and the initialization is provided in the simulation API's run method.

To simplify this process, a feature has been added to save the simulation outputs required to generate the Metrics inputs and a method to reload those outputs as inputs.

In [2]:
sim = Simulation("DINWOODIE", "base.yaml")

# Both of these parameters are True by default for convenience
sim.run(create_metrics=True, save_metrics_inputs=True)

# Load the metrics data
fpath = sim.env.metrics_input_fname.parent
fname = sim.env.metrics_input_fname.name
metrics = Metrics.from_simulation_outputs(fpath, fname)

# Delete the log files now that they're loaded in
sim.env.cleanup_log_files(log_only=False)

# Alternatively, in this case because the simulation was run, we can use the
# following for convenience convenience only
metrics = sim.metrics

## Availability

There are two methods to produce availability, which have their own function calls:
 - energy: `production_based_availability`
 - time: `time_based_availability`

Here, we will go through the various input definitions to get time-based availability data as both methods use the same inputs, and provide outputs in the same format.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [3]:
# Project total at the whole windfarm level
total = metrics.time_based_availability(frequency="project", by="windfarm")
print(f"Project total: {total * 100:.1f}%")

Project total: 96.2%


In [4]:
# Project total at the turbine level
metrics.time_based_availability(frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.96,0.93,0.98,0.97,0.98,0.92,0.92,0.96,0.98,0.93,0.98,0.88,0.98,0.98,0.99,0.98,0.98,0.98,0.96,0.95,0.98,0.96,0.98,0.98,0.98,0.93,0.98,0.98,0.96,0.97,0.97,0.91,0.98,0.90,0.96,0.98,0.95,0.98,0.98,0.98,0.98,0.98,0.95,0.96,0.98,0.98,0.98,0.89,0.89,0.95,0.98,0.98,0.91,0.93,0.98,0.98,0.94,0.98,0.98,0.94,0.97,0.95,0.98,0.89,0.96,0.98,0.94,0.98,0.98,0.98,0.98,0.98,0.98,0.93,0.93,0.97,0.98,0.95,0.98,0.98


In [5]:
# Project annual totals at the windfarm level
metrics.time_based_availability(frequency="annual", by="windfarm")

,windfarm
year,
2003,0.97
2004,0.95
2005,0.94
2006,0.97
2007,0.97
2008,0.96
2009,0.96
2010,0.96
2011,0.97


In [6]:
# Project monthly totals at the windfarm level
metrics.time_based_availability(frequency="monthly", by="windfarm")

,windfarm
month,
1,0.96
2,0.97
3,0.97
4,0.97
5,0.96
6,0.96
7,0.96
8,0.97
9,0.96


In [7]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.time_based_availability(frequency="month-year", by="windfarm").head(24)

windfarm
year month          
2003 1          0.98
     2          0.95
     3          0.96
     4          0.96
     5          0.99
     6          0.99
     7          0.99
     8          0.98
     9          0.96
     10         0.95
     11         0.95
     12         0.96
2004 1          0.97
     2          0.97
     3          0.95
     4          0.95
     5          0.94
     6          0.94
     7          0.95
     8          0.97
     9          0.97
     10         0.95
     11         0.93
     12         0.92

## Capacity Factor

Here, we will go through the various input definitions to get capacity factor data. The inputs are very similar to that of the availability calculation.

`which` options:
 - net: net capcity factor, actual production
 - gross: gross capacity factor, potential production

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [8]:
# Project total at the whole windfarm level
cf = metrics.capacity_factor(which="net", frequency="project", by="windfarm")
print(f"  Net Capacity Factor: {cf:.2f}%")

cf = metrics.capacity_factor(which="gross", frequency="project", by="windfarm")
print(f"Gross Capacity Factor: {cf:.2f}%")

  Net Capacity Factor: 0.46%
Gross Capacity Factor: 0.48%


In [9]:
# Project total at the turbine level
metrics.capacity_factor(which="net", frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.46,0.44,0.47,0.46,0.47,0.44,0.44,0.46,0.47,0.44,0.47,0.41,0.47,0.47,0.47,0.46,0.47,0.47,0.46,0.45,0.47,0.46,0.47,0.47,0.47,0.44,0.47,0.46,0.46,0.47,0.46,0.43,0.47,0.43,0.46,0.47,0.46,0.47,0.46,0.47,0.47,0.47,0.45,0.45,0.47,0.47,0.47,0.43,0.42,0.46,0.47,0.47,0.43,0.44,0.47,0.47,0.45,0.47,0.47,0.46,0.46,0.46,0.47,0.42,0.46,0.47,0.45,0.47,0.47,0.47,0.46,0.47,0.47,0.44,0.45,0.46,0.46,0.46,0.47,0.46


In [10]:
# Project annual totals at the windfarm level
metrics.capacity_factor(which="net", frequency="annual", by="windfarm")

,windfarm
year,
2003,0.41
2004,0.44
2005,0.46
2006,0.45
2007,0.50
2008,0.50
2009,0.46
2010,0.41
2011,0.49


In [11]:
# Project monthly totals at the windfarm level
metrics.capacity_factor(which="net", frequency="monthly", by="windfarm")

,windfarm
month,
1,0.60
2,0.51
3,0.51
4,0.41
5,0.39
6,0.32
7,0.33
8,0.36
9,0.46


In [12]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.capacity_factor(which="net", frequency="month-year", by="windfarm").head(24)

windfarm
year month          
2003 1          0.62
     2          0.35
     3          0.37
     4          0.52
     5          0.35
     6          0.31
     7          0.30
     8          0.30
     9          0.27
     10         0.42
     11         0.53
     12         0.54
2004 1          0.54
     2          0.51
     3          0.50
     4          0.39
     5          0.28
     6          0.37
     7          0.26
     8          0.41
     9          0.58
     10         0.55
     11         0.46
     12         0.46

## Task Completion Rate

Here, we will go through the various input definitions to get the task completion rates. The inputs are very similar to that of the availability calculation.

`which` options:
 - scheduled: scheduled maintenance only (classified as maintenace tasks in inputs)
 - unscheduled: unscheduled maintenance only (classified as failure events in inputs)
 - both: 

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis

In [13]:
# Project total at the whole windfarm level
total = metrics.task_completion_rate(which="scheduled", frequency="project")
print(f"  Scheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="unscheduled", frequency="project")
print(f"Unscheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="both", frequency="project")
print(f"    Overall Task Completion Rate: {total * 100:.0f}%")

  Scheduled Task Completion Rate: 100%
Unscheduled Task Completion Rate: 100%
    Overall Task Completion Rate: 100%


In [14]:
# Project annual totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="annual")

,Completion Rate
year,
2003,0.99
2004,0.96
2005,1.03
2006,1.01
2007,1.00
2008,1.00
2009,1.00
2010,1.00
2011,1.00


In [15]:
# Project monthly totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="monthly")

,Completion Rate
month,
1,0.94
2,1.10
3,1.00
4,1.03
5,0.99
6,1.00
7,1.03
8,0.96
9,1.02


In [16]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.task_completion_rate(which="both", frequency="month-year").head(24)

Completion Rate
year month                 
2003 1                 0.86
     2                 1.09
     3                 1.01
     4                 1.05
     5                 1.02
     6                 0.98
     7                 0.97
     8                 1.02
     9                 0.86
     10                1.11
     11                1.00
     12                0.98
2004 1                 0.50
     2                 1.23
     3                 1.28
     4                 1.16
     5                 1.07
     6                 1.12
     7                 1.04
     8                 1.03
     9                 0.97
     10                0.94
     11                0.92
     12                0.55

## Equipment Costs

Here, we will go through the various input definitions to get the equipment cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_equipment` options:
 - `True`: computed across all equipment used
 - `False`: computed for each piece of equipment

In [17]:
# Project total at the whole windfarm level
total = metrics.equipment_costs(frequency="project", by_equipment=False)
print(f"Project total: ${total / metrics.project_capacity:,.2f}/MW")

Project total: $440,403.05/MW


In [18]:
# Project totals at the equipment level
metrics.equipment_costs(frequency="project", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,"6,393,260.42","6,393,260.42","6,393,260.42","1,373,101.08","85,143,848.96"


In [19]:
# Project annual totals at the windfarm level
metrics.equipment_costs(frequency="annual", by_equipment=False)

,equipment_cost
year,
2003,"18,922,321.18"
2004,"9,091,975.02"
2005,"12,179,640.00"
2006,"11,125,352.43"
2007,"7,479,753.47"
2008,"5,318,097.22"
2009,"8,839,478.51"
2010,"12,549,812.50"
2011,"12,817,532.73"


In [20]:
# Project monthly totals at the equipment level
metrics.equipment_costs(frequency="monthly", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
month,,,,,
1,"543,010.42","543,010.42","543,083.33","81,812.37","14,647,916.67"
2,"496,125.00","496,125.00","496,052.08","546,678.16","4,998,333.33"
3,"541,989.58","541,625.00","541,989.58","99,750.00","956,250.00"
4,"524,635.42","525,000.00","524,635.42",0.00,"4,667,291.67"
5,"542,500.00","542,500.00","542,500.00",0.00,"5,435,444.44"
6,"525,000.00","525,000.00","525,000.00",0.00,"8,458,800.35"
7,"542,500.00","542,500.00","542,500.00",0.00,"18,083,498.26"
8,"542,500.00","542,500.00","542,500.00","121,963.29","4,881,250.00"
9,"525,000.00","525,000.00","525,000.00","161,839.65",0.00


In [21]:
# Project month-by-year totals at the equipment level
# NOTE: This is limited to the two years only
metrics.equipment_costs(frequency="month-year", by_equipment=True).head(24)

Crew Transfer Vessel 1  Crew Transfer Vessel 2  \
year month                                                   
2003 1                   54,760.42               54,760.42   
     2                   49,000.00               49,000.00   
     3                   54,250.00               54,250.00   
     4                   52,500.00               52,500.00   
     5                   54,250.00               54,250.00   
     6                   52,500.00               52,500.00   
     7                   54,250.00               54,250.00   
     8                   54,250.00               54,250.00   
     9                   52,500.00               52,500.00   
     10                  54,250.00               54,250.00   
     11                  52,500.00               52,500.00   
     12                  54,250.00               54,250.00   
2004 1                   54,250.00               54,250.00   
     2                   50,750.00               50,750.00   
     3                   54,250.00               54,250.00   
     4                   52,500.00               52,500.00   
     5                   54,250.00               54,250.00   
     6                   52,500.00               52,500.00   
     7                   54,250.00               54,250.00   
     8                   54,250.00               54,250.00   
     9                   52,500.00               52,500.00   
     10                  54,250.00               54,250.00   
     11                  52,500.00               52,500.00   
     12                  54,250.00               54,250.00   

            Crew Transfer Vessel 3  Field Support Vessel  Heavy Lift Vessel  
year month                                                                   
2003 1                   54,760.42                  0.00               0.00  
     2                   49,000.00                  0.00               0.00  
     3                   54,250.00                  0.00               0.00  
     4                   52,500.00                  0.00       4,667,291.67  
     5                   54,250.00                  0.00       1,031,250.00  
     6                   52,500.00                  0.00       4,014,366.32  
     7                   54,250.00                  0.00       1,050,000.00  
     8                   54,250.00                  0.00               0.00  
     9                   52,500.00                  0.00               0.00  
     10                  54,250.00                  0.00               0.00  
     11                  52,500.00                  0.00       2,166,631.94  
     12                  54,250.00                  0.00       4,075,000.00  
2004 1                   54,250.00             51,803.15               0.00  
     2                   50,750.00            218,500.00               0.00  
     3                   54,250.00                  0.00               0.00  
     4                   52,500.00                  0.00               0.00  
     5                   54,250.00                  0.00               0.00  
     6                   52,500.00                  0.00               0.00  
     7                   54,250.00                  0.00       4,161,824.65  
     8                   54,250.00                  0.00         900,000.00  
     9                   52,500.00                  0.00               0.00  
     10                  54,250.00                  0.00               0.00  
     11                  52,500.00                  0.00               0.00  
     12                  54,250.00                  0.00       1,838,347.22

## Service Equipment Utilization Rate

Here, we will go through the various input definitions to get the service equipment utiliztion rates.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis

In [22]:
# Project totals at the project level
total = metrics.service_equipment_utilization(frequency="project")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,1.00,1.00,1.00,0.90,0.95


In [23]:
# Annualized project totals
total = metrics.service_equipment_utilization(frequency="annual")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
year,,,,,
2003,1.00,1.00,1.00,0.00,0.97
2004,1.00,1.00,1.00,0.94,0.93
2005,1.00,1.00,1.00,0.92,0.93
2006,1.00,1.00,1.00,0.00,0.95
2007,1.00,1.00,1.00,0.00,0.97
2008,1.00,1.00,1.00,0.00,0.95
2009,1.00,1.00,1.00,0.94,0.91
2010,1.00,1.00,1.00,0.86,0.98
2011,1.00,1.00,1.00,0.94,0.96


## Labor Costs

Here, we will go through the various input definitions to get the labor cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_type` options:
 - `True`: computed across each labor type
 - `False`: computed for both labor types used

In [24]:
# Project total at the whole windfarm level
total = metrics.labor_costs(frequency="project", by_type=False)
print(f"Project total: ${total / metrics.project_capacity:,.2f}/MW")

Project total: $0.00/MW


In [25]:
# Project totals for each type of labor
# NOTE: Only salaried labor was defined for thesese analyses
metrics.labor_costs(frequency="project", by_type=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost
0,0.00,0.00,0.00


In [26]:
# Project annual totals for all labor
metrics.labor_costs(frequency="annual", by_type=False)

,total_labor_cost
year,
2003,0.00
2004,0.00
2005,0.00
2006,0.00
2007,0.00
2008,0.00
2009,0.00
2010,0.00
2011,0.00


In [27]:
# Project monthly totals for all labor
metrics.labor_costs(frequency="monthly", by_type=False)

,total_labor_cost
month,
1,0.00
2,0.00
3,0.00
4,0.00
5,0.00
6,0.00
7,0.00
8,0.00
9,0.00


In [28]:
# Project month-by-year totals for all labor
# NOTE: This is limited to the first two years only
metrics.labor_costs(frequency="month-year", by_type=False).head(24)

total_labor_cost
year month                  
2003 1                  0.00
     2                  0.00
     3                  0.00
     4                  0.00
     5                  0.00
     6                  0.00
     7                  0.00
     8                  0.00
     9                  0.00
     10                 0.00
     11                 0.00
     12                 0.00
2004 1                  0.00
     2                  0.00
     3                  0.00
     4                  0.00
     5                  0.00
     6                  0.00
     7                  0.00
     8                  0.00
     9                  0.00
     10                 0.00
     11                 0.00
     12                 0.00

## Equipment and Labor Costs

Here, we will go through the various input definitions to get the equipment and labor cost data broken out by expense categories.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across as equipment, plus each labor type, plus totals
 - `False`: computed as single total
 
**NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`reason` definitions:
 - Maintenance: routine maintenance
 - Repair: unscheduled maintenance, ranging from inspections to replacements
 - Weather Delay: Any delays caused by unsafe weather conditions
 - No Requests: Equipment and labor is active, but there are no repairs or maintenance tasks to be completed
 - Not in Shift: Any time outside of the operating hours of the windfarm

In [29]:
# Project totals
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=False)

,total_cost
reason,
Maintenance,"2,380,692.71"
Repair,"18,967,151.04"
Crew Transfer,"1,081,098.97"
Site Travel,0.00
Mobilization,"8,000,000.00"
Weather Delay,"34,965,416.12"
No Requests,"20,979,470.49"
Not in Shift,"19,322,901.97"


In [30]:
# Project totals by each category
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost,equipment_cost,total_cost
reason,,,,,
Maintenance,0.00,0.00,0.00,"2,380,692.71","2,380,692.71"
Repair,0.00,0.00,0.00,"18,967,151.04","18,967,151.04"
Crew Transfer,0.00,0.00,0.00,"1,081,098.97","1,081,098.97"
Site Travel,0.00,0.00,0.00,0.00,0.00
Mobilization,0.00,0.00,0.00,"8,000,000.00","8,000,000.00"
Weather Delay,0.00,0.00,0.00,"34,965,416.12","34,965,416.12"
No Requests,0.00,0.00,0.00,"20,979,470.49","20,979,470.49"
Not in Shift,0.00,0.00,0.00,"19,322,901.97","19,322,901.97"


In [31]:
# Project annual totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="annual", by_category=False).head(10)

total_cost
year reason                    
2003 Maintenance      57,804.69
     Repair        2,805,006.94
     Crew Transfer   125,916.67
     Site Travel           0.00
     Mobilization  1,500,000.00
     Weather Delay 6,136,468.75
     No Requests   5,807,194.44
     Not in Shift  2,489,929.69
2004 Maintenance     452,138.02
     Repair        1,437,274.31

In [32]:
# Project monthly totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="monthly", by_category=False).head(10)

total_cost
month reason                    
1     Maintenance     186,174.48
      Repair        2,786,192.71
      Crew Transfer   138,096.35
      Site Travel           0.00
      Mobilization    500,000.00
      Weather Delay 8,175,536.46
      No Requests   1,964,940.10
      Not in Shift  2,607,893.10
2     Maintenance     158,958.33
      Repair        1,481,622.40

In [33]:
# Project month-by-year totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="month-year", by_category=False).head(20)

total_cost
year month reason                   
2003 1     Maintenance          0.00
           Repair          25,703.12
           Crew Transfer    2,770.83
           Site Travel          0.00
           Mobilization         0.00
           Weather Delay   20,635.42
           No Requests    102,229.17
           Not in Shift    12,942.71
     2     Maintenance          0.00
           Repair          26,651.04
           Crew Transfer    3,098.96
           Site Travel          0.00
           Mobilization         0.00
           Weather Delay   15,531.25
           No Requests     81,375.00
           Not in Shift    20,343.75
     3     Maintenance          0.00
           Repair          21,838.54
           Crew Transfer    2,697.92
           Site Travel          0.00

## Component

Here, we will go through the various input definitions to get the component cost data broken out by various categories.

**NOTE**: It should be noted that the the component costs will not sum up to the whole project operations costs because of delays that are not associated with any repair or maintenance task, such as no requests needing to be processed.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across each cost category (includes total)
 - `False`: computed as single total
 
 `by_action` options:
 - `True`: computed by each of "repair", "maintenance", and "delay"
 - `False`: computed as single total
 
**NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`action` definitions:
 - maintenance: routine maintenance
 - repair: unscheduled maintenance, ranging from inspections to replacements
 - delay: Any delays caused by unsafe weather conditions or not being able to finish a process within a single shift

In [34]:
# Project totals by component
metrics.component_costs(frequency="project", by_category=False, by_action=False)

,total_cost
component,
turbine,"111,799,668.48"


In [35]:
# Project totals by each category and action type
metrics.component_costs(frequency="project", by_category=True, by_action=True)

materials_cost  total_labor_cost  equipment_cost  \
component action                                                          
turbine   delay                  0.00              0.00   54,148,725.76   
          maintenance            0.00              0.00    2,380,692.71   
          repair                 0.00              0.00   18,967,151.04   

                         total_cost  
component action                     
turbine   delay       54,148,725.76  
          maintenance  2,380,692.71  
          repair      18,967,151.04

In [36]:
# Project annual totals by category
# NOTE: This is limited to the first two years
metrics.component_costs(frequency="annual", by_category=True, by_action=False).head(28)

,,materials_cost,total_labor_cost,equipment_cost,total_cost
year,component,,,,
2003,turbine,"3,548,500.00",0.00,"11,607,470.49","15,155,970.49"
2004,turbine,"3,661,000.00",0.00,"6,031,339.41","9,692,339.41"
2005,turbine,"4,076,500.00",0.00,"10,494,289.93","14,570,789.93"
2006,turbine,"4,384,000.00",0.00,"7,860,312.50","12,244,312.50"
2007,turbine,"2,980,500.00",0.00,"6,220,453.99","9,200,953.99"
2008,turbine,"2,499,500.00",0.00,"4,129,690.97","6,629,190.97"
2009,turbine,"3,321,500.00",0.00,"5,873,780.38","9,195,280.38"
2010,turbine,"4,726,000.00",0.00,"10,743,760.42","15,469,760.42"
2011,turbine,"3,085,000.00",0.00,"9,426,040.54","12,511,040.54"


In [37]:
# Project monthly totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="monthly", by_category=True, by_action=False).head(28)

,,materials_cost,total_labor_cost,equipment_cost,total_cost
month,component,,,,
1,turbine,"4,625,500.00",0.00,"13,879,549.48","18,505,049.48"
2,turbine,"3,103,000.00",0.00,"5,248,638.89","8,351,638.89"
3,turbine,"1,873,500.00",0.00,"1,827,677.08","3,701,177.08"
4,turbine,"2,166,500.00",0.00,"4,989,463.54","7,155,963.54"
5,turbine,"2,871,000.00",0.00,"5,340,874.13","8,211,874.13"
6,turbine,"2,104,500.00",0.00,"3,787,574.65","5,892,074.65"
7,turbine,"5,564,500.00",0.00,"13,547,464.41","19,111,964.41"
8,turbine,"2,990,000.00",0.00,"3,672,351.31","6,662,351.31"
9,turbine,"1,334,500.00",0.00,"942,742.19","2,277,242.19"


In [38]:
# Project month-by-year totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="month-year", by_category=True, by_action=False).head(28)

materials_cost  total_labor_cost  equipment_cost  \
year month component                                                     
2003 1     turbine        106,500.00              0.00       59,645.83   
     2     turbine        106,500.00              0.00       64,750.00   
     3     turbine         35,500.00              0.00       33,395.83   
     4     turbine      1,058,500.00              0.00    4,206,302.08   
     5     turbine        371,000.00              0.00    1,058,703.13   
     6     turbine         45,500.00              0.00       27,562.50   
     7     turbine         38,500.00              0.00       37,697.92   
     8     turbine         34,500.00              0.00       34,434.90   
     9     turbine         59,000.00              0.00       70,802.08   
     10    turbine         96,000.00              0.00       76,270.83   
     11    turbine        485,000.00              0.00    1,755,298.61   
     12    turbine      1,112,000.00              0.00    4,182,606.77   
2004 1     turbine        174,500.00              0.00      208,244.79   
     2     turbine        413,000.00              0.00      208,916.67   
     3     turbine        283,000.00              0.00      161,000.00   
     4     turbine        224,500.00              0.00      154,875.00   
     5     turbine        256,500.00              0.00      160,125.00   
     6     turbine        231,500.00              0.00      153,125.00   
     7     turbine      1,238,500.00              0.00    2,969,619.79   
     8     turbine        205,000.00              0.00      127,877.60   
     9     turbine        110,500.00              0.00       96,632.81   
     10    turbine        183,500.00              0.00      135,825.52   
     11    turbine        142,500.00              0.00      155,750.00   
     12    turbine        198,000.00              0.00    1,499,347.22   
2005 1     turbine        762,000.00              0.00    4,360,312.50   
     2     turbine        183,500.00              0.00      144,484.38   
     3     turbine         70,500.00              0.00       71,531.25   
     4     turbine         54,000.00              0.00       43,786.46   

                       total_cost  
year month component               
2003 1     turbine     166,145.83  
     2     turbine     171,250.00  
     3     turbine      68,895.83  
     4     turbine   5,264,802.08  
     5     turbine   1,429,703.13  
     6     turbine      73,062.50  
     7     turbine      76,197.92  
     8     turbine      68,934.90  
     9     turbine     129,802.08  
     10    turbine     172,270.83  
     11    turbine   2,240,298.61  
     12    turbine   5,294,606.77  
2004 1     turbine     382,744.79  
     2     turbine     621,916.67  
     3     turbine     444,000.00  
     4     turbine     379,375.00  
     5     turbine     416,625.00  
     6     turbine     384,625.00  
     7     turbine   4,208,119.79  
     8     turbine     332,877.60  
     9     turbine     207,132.81  
     10    turbine     319,325.52  
     11    turbine     298,250.00  
     12    turbine   1,697,347.22  
2005 1     turbine   5,122,312.50  
     2     turbine     327,984.38  
     3     turbine     142,031.25  
     4     turbine      97,786.46

## Fixed Cost Impacts

Here, we will go through the various input definitions to get the fixed cost data

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 
`resolution` options:
 - high: computed across the lowest itemized cost levels
 - medium: computed across overarching cost levels
 - low: computed as single total

In [39]:
# The resolution hierarchy for fixed costs
pprint(metrics.fixed_costs.hierarchy)

{'operations': {'annual_leases_fees': ['submerge_land_lease_costs',
                                       'transmission_charges_rights'],
                'environmental_health_safety_monitoring': [],
                'insurance': ['brokers_fee',
                              'operations_all_risk',
                              'business_interruption',
                              'third_party_liability',
                              'storm_coverage'],
                'labor': [],
                'onshore_electrical_maintenance': [],
                'operating_facilities': [],
                'operations_management_administration': ['project_management_administration',
                                                         'marine_management',
                                                         'weather_forecasting',
                                                         'condition_monitoring']}}


In [40]:
# Project totals at the highest level
metrics.project_fixed_costs(frequency="project", resolution="low")

,operations
0,"16,000,080.00"


In [41]:
# Project totals at the medium level
metrics.project_fixed_costs(frequency="project", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,"16,000,080.00"


In [42]:
# Project totals at the lowest level
metrics.project_fixed_costs(frequency="project", resolution="high")

,project_management_administration,marine_management,weather_forecasting,condition_monitoring,brokers_fee,operations_all_risk,business_interruption,third_party_liability,storm_coverage,submerge_land_lease_costs,transmission_charges_rights,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"16,000,080.00"


In [43]:
# Project annualized totals at the medium level
metrics.project_fixed_costs(frequency="annual", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
year,,,,,,,
2003,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2004,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2005,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2006,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2007,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2008,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2009,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2010,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2011,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"


## Process Times

There are no inputs for the process timing as it is a slow calculation, so aggregation is left to the user for now. The results corresond to the number of hours required to complete any of the repair or maintenance activities.

In [44]:
# Project totals at the project level
total = metrics.process_times()
total

,time_to_completion,process_time,downtime,N
category,,,,
annual service,"229,063.16","79,434.25","71,520.25",546.00
major repair,"73,413.38","1,558.68","1,131.80",22.00
major replacement,"157,353.82","10,302.62","157,353.82",55.00
manual reset,"215,546.27","37,435.00","13,396.75","5,635.00"
medium repair,"13,543.95","10,443.25","8,137.75",207.00
minor repair,"75,208.81","31,352.50","14,263.50","2,300.00"


## Power Production

Here, we will go through the various input definitions to get the power production data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_turbine` options:
 - `True`: computed for each turbines
 - `False`: computed for the whole windfarm

In [45]:
# Project total at the whole windfarm level
total = metrics.power_production(frequency="project", by_turbine=False)
total

,windfarm
Project Energy Production (kWh),"9,644,674,062.50"


In [46]:
# Project totals at the turbine level
metrics.power_production(frequency="project", by_turbine=True)

,windfarm,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
Project Energy Production (kWh),"9,644,674,062.50","119,762,540.50","115,932,554.50","122,948,337.50","122,090,021.00","122,725,086.50","114,506,500.00","114,609,779.00","121,180,912.00","122,467,275.00","116,129,895.00","122,956,961.00","108,900,020.00","123,621,744.50","122,538,708.50","123,480,636.50","121,929,272.50","123,334,499.50","123,440,503.00","120,925,865.50","118,221,805.00","122,460,267.00","120,082,886.50","123,808,983.00","122,678,048.50","122,961,776.00","116,694,224.50","122,561,157.00","121,805,823.00","121,223,449.50","122,461,235.00","121,990,930.50","112,987,397.00","123,073,424.00","111,946,870.00","121,259,685.50","123,724,165.00","120,300,312.00","123,382,171.50","122,212,482.00","122,932,996.00","123,247,527.50","122,547,522.50","119,664,962.00","119,571,427.50","122,961,741.50","123,040,326.00","123,876,944.50","113,861,692.00","111,404,579.50","119,929,312.00","122,780,138.50","122,598,464.50","114,371,999.00","116,000,878.50","122,340,772.00","123,388,824.00","118,418,372.00","122,565,180.50","122,907,427.00","120,235,811.00","121,635,622.50","119,824,599.00","123,435,054.00","110,582,592.00","119,695,408.00","122,436,150.50","117,883,161.50","122,737,135.00","123,289,862.50","123,572,392.00","122,198,493.00","123,194,813.00","123,425,598.00","115,791,509.50","117,593,426.00","121,692,833.50","121,447,250.50","120,465,785.50","123,713,426.50","122,093,847.50"


In [47]:
# Project annual totals for the windfarm
metrics.power_production(frequency="annual", by_turbine=False)

,windfarm
year,
2003,"855,792,902.50"
2004,"932,333,753.00"
2005,"968,927,756.50"
2006,"954,497,995.50"
2007,"1,040,748,585.50"
2008,"1,056,618,234.50"
2009,"959,096,330.50"
2010,"852,219,668.50"
2011,"1,028,287,564.00"


In [48]:
# Project monthly totals for the windfarm
metrics.power_production(frequency="monthly", by_turbine=False)

,windfarm
month,
1,"1,074,148,274.50"
2,"825,404,792.00"
3,"907,169,443.00"
4,"713,299,417.50"
5,"687,982,612.00"
6,"556,191,765.00"
7,"588,499,462.50"
8,"647,670,217.00"
9,"797,240,416.00"


In [49]:
# Project month-by-year totals for the windfarm
# NOTE: This is limited to the first two years only
metrics.power_production(frequency="month-year", by_turbine=False).head(24)

windfarm
year month               
2003 1     110,543,210.00
     2      56,946,622.00
     3      65,308,746.00
     4      90,301,748.00
     5      62,278,100.00
     6      53,997,918.00
     7      53,887,505.50
     8      53,067,963.00
     9      46,430,973.00
     10     74,437,725.50
     11     91,603,661.50
     12     96,988,730.00
2004 1      96,861,238.00
     2      85,127,902.00
     3      89,766,058.50
     4      66,596,132.00
     5      50,648,130.50
     6      64,176,729.00
     7      47,013,916.50
     8      72,969,110.00
     9      99,687,206.00
     10     97,346,366.00
     11     79,669,698.00
     12     82,471,266.50

## PySAM-Powered Results

For a number of project financial metrics, the PySAM library is utilized.

<div class="alert alert-block alert-warning">
<b>NOTE:</b> If a "SAM_settings" file is not provided to the simulation, then the following metrics will not be able to be calculated and will raise a `NotImplementedError`.
</div>

With the above warning in mind, the appropriate simulation outputs are provided as inputs to PySAM upon initialization to ensure all values are aligned.

### Net Present Value (NPV)

In [50]:
try:
    npv = metrics.pysam_npv()
    print(f"NPV: ${npv:,.0f}")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_npv()' cannot be calculated!


### Real Levelized Cost of Energy (LCOE)

In [51]:
try:
    lcoe = metrics.pysam_lcoe_real()
    print(f"Real LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_real()' cannot be calculated!


### Nominal Levelized Cost of Energy (LCOE)

In [52]:
try:
    lcoe = metrics.pysam_lcoe_nominal()
    print(f"Nominal LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_nominal()' cannot be calculated!


### After-tax Internal Return Rate (IRR)

In [53]:
try:
    npv = metrics.pysam_irr()
    print(f"IRR: {npv:,.1f}%")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_irr()' cannot be calculated!


### One Data Frame to Rule Them All

For this demonstration we will manually load a PySAM settings file and trigger the setup for demonstration purposes, but it should be noted that this practice should be avoided.

In [54]:
try:
    metrics.pysam_all_outputs()
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_all_outputs()' cannot be calculated!
